![PPGI_UFRJ](https://github.com/zavaleta/Fundamentos_DS/blob/main/imagens/ppgi-ufrj.png?raw=1)
# **Fundamentos de Ciência de Dados**

> Trabalho Final - 2º Período de 2022

### **Disponibilizando Dados sobre Resultados Financeiros de Cias Abertas Enriquecidos com Proveniência para a [OBInvest](https://obinvest.org/)**

---

**Alunos:** Gilberto Gil | Saulo Andrade Almeida | Valquire Jesus



## **FAIRificação**

In [118]:
#checking version machine architecture, OS, python and all libs used in this notebook
import platform
import os
import numpy
import pandas
import conda
import pydot
import prov
import seaborn
import matplotlib
import plotly

def verifica_versoes_ambiente_bibliotecas(details=False):

    #definnig version of python and all libs used
    HOST_MACHINE_ARCHTECTURE_EXPECTED = 'x86_64'
    HOST_MACHINE_OS_EXPECTED = 'Linux'
    CONDA_VERSION_EXPECTED = '4.14.0'
    CONDA_DEFAULT_ENV_EXPECTED = 'drecvmenv'
    PYTHON_VERSION_EXPECTED = '3.9.12'
    NUMPY_LIB_VERSION_EXPECTED = '1.21.5'
    PANDAS_LIB_VERSION_EXPECTED = '1.4.2'
    PYDOT_LIB_VERSION_EXPECTED = '1.4.2'
    PROV_LIB_VERSION_EXPECTED = '2.0.0'
    SEABORN_LIB_VERSION_EXPECTED = '0.11.2'
    MATPLOTLIB_LIB_VERSION_EXPECTED = '3.5.1'
    PLOTLY_LIB_VERSION_EXPECTED = '5.6.0'
    
    

    if details: 
        print('Host Machine Architecture:', platform.machine())
        print('Host Machine OS:', platform.system())
        print('Conda Version:', conda.__version__)
        print('Conda default env:', os.environ['CONDA_DEFAULT_ENV'])
        print('Python Version:', platform.python_version())
        print('NumPy Lib Version:', numpy.__version__)
        print('Pandas Lib Version:', pandas.__version__)
        print('PyDot Lib Version:', pydot.__version__)
        print('Prov Lib Version:', prov.__version__)
        print('Seaborn Lib Version:', seaborn.__version__)
        print('Matplotlib Lib Version:', matplotlib.__version__)
        print('Plotly Lib Version:', plotly.__version__)
        
        
    #checking versions
    try:
        #checking Machine Architecute expected
        assert platform.machine() == HOST_MACHINE_ARCHTECTURE_EXPECTED

        #checking OS expected
        assert platform.system() == HOST_MACHINE_OS_EXPECTED
        
        #checking conda version
        assert conda.__version__ == CONDA_VERSION_EXPECTED
        
        #checking conda default environment
        assert os.environ['CONDA_DEFAULT_ENV'] == CONDA_DEFAULT_ENV_EXPECTED    

        #checking python version
        assert platform.python_version() == PYTHON_VERSION_EXPECTED

        #checking numpy lib version
        assert numpy.__version__ == NUMPY_LIB_VERSION_EXPECTED  

        #checking Pandas lib version
        assert pandas.__version__ == PANDAS_LIB_VERSION_EXPECTED
        
        #checking pydot version
        assert pydot.__version__ == PYDOT_LIB_VERSION_EXPECTED
        
        #checking prov version
        assert prov.__version__ == PROV_LIB_VERSION_EXPECTED
        
        #checking seaborn version
        assert seaborn.__version__ == SEABORN_LIB_VERSION_EXPECTED
        
        #checking matplotlib version
        assert matplotlib.__version__ == MATPLOTLIB_LIB_VERSION_EXPECTED
        
        #checking plotly version
        assert plotly.__version__ == PLOTLY_LIB_VERSION_EXPECTED
    except:
        #if any assert fail, or something else get wrong during verification
        if details: print('Alguma coisa errada. Favor verificar versoes do ambiente e das bibpliotecas!')
        return False
    else:
        #if pass all asserts
        if details: print('Todas as versoes do ambiente e bibliotecas estao corretas!')
        return True

## **Pré-processamento de Dados**

In [119]:
#Utilizado ambiente python 3.9
import pandas as pd
import numpy as np
import sys, subprocess, datetime

def carrega_csv(path, nomes_colunas_data=[]):
    return pd.read_csv(path, sep=';', header=0, encoding="ISO-8859-1", parse_dates=nomes_colunas_data)

def carrega_csvs_dre(template_path_arquivo, anos):
    tempDF = pd.DataFrame() 
    
    for ano in anos:
        df_ano_corrente = carrega_csv(template_path_arquivo.format(ano), 
                                      ['DT_REFER', 'DT_INI_EXERC', 'DT_FIM_EXERC'])
        
        tempDF = pd.concat([tempDF, df_ano_corrente] , ignore_index=True)
        
    return tempDF
    

def carregar_dados_empresas():
    return carrega_csv("data/cad-emp/cad_cia_aberta.csv")

def lista_todos_anos():
    return [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
    #return [2020, 2021]

def carrega_todos_anos_dre_itr():
    return carrega_csvs_dre('data/dre-itr/itr_cia_aberta_DRE_con_{0}.csv', lista_todos_anos())

def carrega_todos_anos_dre_dfp():
    return carrega_csvs_dre('data/dre-dfp/dfp_cia_aberta_DRE_con_{0}.csv', lista_todos_anos())

def carregar_datasets(documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades):
    
    #Salvar hora do inicio da execucao
    inicioExec = datetime.datetime.now()
    
    companyInfoDF = carregar_dados_empresas()
    dreItrDF = carrega_todos_anos_dre_itr()
    dreDfpDF = carrega_todos_anos_dre_dfp()
    
    #print(companyInfoDF.info())
    #print(dreItrDF.info())
    #print(dreDfpDF.info())
    
    #criar a atividade com hora do final da execucao
    fimExec = datetime.datetime.now()
    dicionario_atividades["at-carregar-ds"] = documento_prov.activity("ufrj:carregar_datasets", inicioExec, fimExec)
    
    #associar a atividade com o agente
    documento_prov.wasAssociatedWith(dicionario_atividades["at-carregar-ds"], dicionario_agentes["ag-drecvm-ipynb"])
    
    #associar a atividade com o dataset que foi lido
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-cademp-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dreitr2011-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dreitr2012-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dreitr2013-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dreitr2014-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dreitr2015-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dreitr2016-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dreitr2017-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dreitr2018-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dreitr2019-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dreitr2020-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dreitr2021-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dredfp2011-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dredfp2012-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dredfp2013-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dredfp2014-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dredfp2015-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dredfp2016-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dredfp2017-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dredfp2018-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dredfp2019-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dredfp2020-ds"])
    documento_prov.used(dicionario_atividades["at-carregar-ds"],dicionario_entidades["ent-dredfp2021-ds"])
    
    #gerar entidade do detaframe e associar a entidade
    dicionario_entidades["ent-cademp-df"] = documento_prov.entity('obinvest:df_cia', {'prov:generatedAtTime': str(fimExec), 'prov:label': 'Dataframe com dados das empresas'})
    dicionario_entidades["ent-dreitr-df"] = documento_prov.entity('obinvest:df_itr', {'prov:generatedAtTime': str(fimExec), 'prov:label': 'Dataframe com dados das DREs trimestrais'})
    dicionario_entidades["ent-dredfp-df"] = documento_prov.entity('obinvest:df_dfp', {'prov:generatedAtTime': str(fimExec), 'prov:label': 'Dataframe com dados das DREs anuais'})
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-cademp-df"], dicionario_atividades["at-carregar-ds"])    
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-dreitr-df"], dicionario_atividades["at-carregar-ds"])    
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-dredfp-df"], dicionario_atividades["at-carregar-ds"])    
    
    return companyInfoDF, dreItrDF, dreDfpDF

def trata_dados_companhias(dataframe_cia, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades):
    
    #Salvar hora do inicio da execucao
    inicioExec = datetime.datetime.now()
    
    # TRATAMENTO DO DF CAD_CIA
    # filtrar as empresas com cadastro ativo - df cad_cia
    dataframe_cia = dataframe_cia[dataframe_cia['SIT'] == 'ATIVO']
    # deletar colunas desnecessárias
    dataframe_cia = dataframe_cia.drop(['DENOM_SOCIAL', 'DENOM_COMERC', 'DT_REG', 'DT_CONST', 'DT_CANCEL', 'MOTIVO_CANCEL', 'SIT', 
                   'DT_INI_SIT', 'TP_MERC', 'CATEG_REG', 'DT_INI_CATEG', 'SIT_EMISSOR', 'DT_INI_SIT_EMISSOR', 
                   'CONTROLE_ACIONARIO', 'TP_ENDER', 'LOGRADOURO', 'COMPL', 'BAIRRO', 'MUN', 'UF', 'PAIS', 
                   'CEP', 'DDD_TEL', 'TEL', 'DDD_FAX', 'FAX', 'EMAIL', 'TP_RESP', 'RESP', 'DT_INI_RESP', 
                   'LOGRADOURO_RESP', 'COMPL_RESP', 'BAIRRO_RESP', 'MUN_RESP', 'UF_RESP', 'PAIS_RESP', 
                   'CEP_RESP', 'DDD_TEL_RESP', 'TEL_RESP', 'DDD_FAX_RESP', 'FAX_RESP', 'EMAIL_RESP', 
                   'CNPJ_AUDITOR', 'AUDITOR'], axis=1)
    
    # deletar os registros duplicados, que possuem mesmos cnpj e códigos cvm
    dataframe_cia = dataframe_cia.drop_duplicates(subset = ['CNPJ_CIA', 'CD_CVM'], keep = 'last')
    
    #criar a atividade com hora do final da execucao
    fimExec = datetime.datetime.now()
    dicionario_atividades["at-trata-dados-companhias"] = documento_prov.activity("ufrj:trata_dados_companhias", inicioExec, fimExec)
    
    #associar a atividade com o agente
    documento_prov.wasAssociatedWith(dicionario_atividades["at-trata-dados-companhias"], dicionario_agentes["ag-drecvm-ipynb"])
    
    #associar a atividade com o dataset que foi lido
    documento_prov.used(dicionario_atividades["at-trata-dados-companhias"],dicionario_entidades["ent-cademp-df"])
    
    #gerar entidade do detaframe e associar a entidade
    dicionario_entidades["ent-cademp-tratado-df"] = documento_prov.entity('obinvest:df_cia_tratado', {'prov:generatedAtTime': str(fimExec), 'prov:label': 'Dataframe com dados das empresas ativas, apenas com os campos necessarios'})
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-cademp-tratado-df"], dicionario_atividades["at-trata-dados-companhias"])    
    
    return dataframe_cia

def adiciona_ano(dataframe_dre):
    ano = []
    
    #adicionando as informacoes da coluna de Ano
    for i in dataframe_dre.itertuples():
        ano.append(i.DT_REFER.year)
    
    dataframe_dre['ANO'] = ano
    
    return dataframe_dre

def adiciona_ano_dres(dataframe_dre_itr, dataframe_dre_dfp, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades):
    
    #Salvar hora do inicio da execucao
    inicioExec = datetime.datetime.now()
    
    dreItr = adiciona_ano(dataframe_dre_itr)
    dreDfp = adiciona_ano(dataframe_dre_dfp)
    
    #criar a atividade com hora do final da execucao
    fimExec = datetime.datetime.now()
    dicionario_atividades["at-adicionar-ano"] = documento_prov.activity("ufrj:adiciona_ano_dres", inicioExec, fimExec)
    
    #associar a atividade com o agente
    documento_prov.wasAssociatedWith(dicionario_atividades["at-adicionar-ano"], dicionario_agentes["ag-drecvm-ipynb"])
    
    #associar a atividade com os dataframes que foram lido
    documento_prov.used(dicionario_atividades["at-adicionar-ano"],dicionario_entidades["ent-dreitr-df"])
    documento_prov.used(dicionario_atividades["at-adicionar-ano"],dicionario_entidades["ent-dredfp-df"])
    
    #gerar entidade do detaframe e associar a atividade
    dicionario_entidades["ent-dreitr-ano-df"] = documento_prov.entity('obinvest:df_itr_ano', {'prov:generatedAtTime': str(fimExec), 'prov:label': 'Dataframe com dados das DREs trimestrais, adicionada a coluna ano'})
    dicionario_entidades["ent-dredfp-ano-df"] = documento_prov.entity('obinvest:df_dfp_ano', {'prov:generatedAtTime': str(fimExec), 'prov:label': 'Dataframe com dados das DREs anuais, adicionada a coluna ano'})
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-dreitr-ano-df"], dicionario_atividades["at-adicionar-ano"])    
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-dredfp-ano-df"], dicionario_atividades["at-adicionar-ano"])      
    
    return dreItr, dreDfp

def merge_dre_itr_setor(dataframe_dre, dataframe_cia, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades):
    #Salvar hora do inicio da execucao
    inicioExec = datetime.datetime.now()

    dfTemp = merge_dre_setor(dataframe_dre, dataframe_cia)
    
    #criar a atividade com hora do final da execucao
    fimExec = datetime.datetime.now()
    dicionario_atividades["at-merge-dreitr-setor"] = documento_prov.activity("ufrj:merge_dre_itr_setor", inicioExec, fimExec)
    
    #associar a atividade com o agente
    documento_prov.wasAssociatedWith(dicionario_atividades["at-merge-dreitr-setor"], dicionario_agentes["ag-drecvm-ipynb"])
    
    #associar a atividade com o dataset que foi lido
    documento_prov.used(dicionario_atividades["at-merge-dreitr-setor"],dicionario_entidades["ent-dreitr-ano-df"])
    documento_prov.used(dicionario_atividades["at-merge-dreitr-setor"],dicionario_entidades["ent-cademp-tratado-df"])
    
    #gerar entidade do detaframe e associar a entidade
    dicionario_entidades["ent-dreitf-setor-df"] = documento_prov.entity('obinvest:df_itr_setor', {'prov:generatedAtTime': str(fimExec), 'prov:label': 'Dataframe com dados das DREs trimestrais, adicionado o campo de setor da empresa'})
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-dreitf-setor-df"], dicionario_atividades["at-merge-dreitr-setor"])    

    return dfTemp

def merge_dre_dfp_setor(dataframe_dre, dataframe_cia, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades):
    #Salvar hora do inicio da execucao
    inicioExec = datetime.datetime.now()

    dfTemp = merge_dre_setor(dataframe_dre, dataframe_cia)
    
    #criar a atividade com hora do final da execucao
    fimExec = datetime.datetime.now()
    dicionario_atividades["at-merge-dredfp-setor"] = documento_prov.activity("ufrj:merge_dre_dfp_setor", inicioExec, fimExec)
    
    #associar a atividade com o agente
    documento_prov.wasAssociatedWith(dicionario_atividades["at-merge-dredfp-setor"], dicionario_agentes["ag-drecvm-ipynb"])
    
    #associar a atividade com o dataset que foi lido
    documento_prov.used(dicionario_atividades["at-merge-dredfp-setor"],dicionario_entidades["ent-dredfp-exerc-df"])
    documento_prov.used(dicionario_atividades["at-merge-dredfp-setor"],dicionario_entidades["ent-cademp-tratado-df"])
    
    #gerar entidade do detaframe e associar a entidade
    dicionario_entidades["ent-dredfp-setor-df"] = documento_prov.entity('obinvest:df_dfp_setor', {'prov:generatedAtTime': str(fimExec), 'prov:label': 'Dataframe com dados das DREs anuais, adicionado o campo de setor da empresa'})
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-dredfp-setor-df"], dicionario_atividades["at-merge-dredfp-setor"])    

    return dfTemp

def merge_dre_setor(dataframe_dre, dataframe_cia):    
    # filtrar os informes da empresas ativas
    dataframe_dre = pd.merge(dataframe_dre, dataframe_cia, how = 'inner', on = ['CNPJ_CIA', 'CD_CVM']).reset_index(drop=True)
    return dataframe_dre.drop(['VERSAO'], axis=1)  # eliminar coluna desnecessária

def filtrar_ultimo_exerc(dre):
    # Descartando resultados do ano anterior e mantendo apenas o último valor 
    return dre[dre['ORDEM_EXERC'] == 'ÚLTIMO']

def filtrar_ultimo_exerc_dfp(dre, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades):
    #Salvar hora do inicio da execucao
    inicioExec = datetime.datetime.now()
    
    dfTemp = filtrar_ultimo_exerc(dre)
    
    #criar a atividade com hora do final da execucao
    fimExec = datetime.datetime.now()
    dicionario_atividades["at-filtrar-exerc-dfp"] = documento_prov.activity("ufrj:filtrar_ultimo_exerc_dfp", inicioExec, fimExec)
    
    #associar a atividade com o agente
    documento_prov.wasAssociatedWith(dicionario_atividades["at-filtrar-exerc-dfp"], dicionario_agentes["ag-drecvm-ipynb"])
    
    #associar a atividade com o dataset que foi lido
    documento_prov.used(dicionario_atividades["at-filtrar-exerc-dfp"],dicionario_entidades["ent-dredfp-ano-df"])
    
    #gerar entidade do detaframe e associar a entidade
    dicionario_entidades["ent-dredfp-exerc-df"] = documento_prov.entity('obinvest:df_dfp_ultimo_exerc', {'prov:generatedAtTime': str(fimExec), 'prov:label': 'Dataframe com dados das DREs anuais apenas do ano relativo ao dataset'})
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-dredfp-exerc-df"], dicionario_atividades["at-filtrar-exerc-dfp"])
    
    return dfTemp
    
def obter_trimestres_123(dataframe_itr, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades):
    
    #Salvar hora do inicio da execucao
    inicioExec = datetime.datetime.now()
    
    # FATIAMENTO DO DF ITR
    dataframe_itr = filtrar_ultimo_exerc(dataframe_itr)
    # OBTENÇÃO DO DF TRIMESTRES 1, 2 E 3
    # criar o df com informações dos trimestres 1 (1 a 3 mês), 2 (4 a 6 mês) e 3 (6 a 9 mês) - df trim123
    dfTemp = dataframe_itr.loc[lambda dataframe_itr: ((dataframe_itr.DT_REFER.dt.month == 3) | 
                            ((dataframe_itr.DT_REFER.dt.month == 6) & (dataframe_itr.DT_INI_EXERC.dt.month > 3)) | 
                            ((dataframe_itr.DT_REFER.dt.month == 9) & (dataframe_itr.DT_INI_EXERC.dt.month > 6))) &
                             (dataframe_itr.DT_REFER.dt.year == dataframe_itr.DT_INI_EXERC.dt.year)]

    #criar a atividade com hora do final da execucao
    fimExec = datetime.datetime.now()
    dicionario_atividades["at-obter-trimestres123"] = documento_prov.activity("ufrj:obter_trimestres_123", inicioExec, fimExec)
    
    #associar a atividade com o agente
    documento_prov.wasAssociatedWith(dicionario_atividades["at-obter-trimestres123"], dicionario_agentes["ag-drecvm-ipynb"])
    
    #associar a atividade com o dataset que foi lido
    documento_prov.used(dicionario_atividades["at-obter-trimestres123"],dicionario_entidades["ent-dreitf-setor-df"])
    
    #gerar entidade do detaframe e associar a entidade
    dicionario_entidades["ent-dreitf-trim123-df"] = documento_prov.entity('obinvest:df_trim123', {'prov:generatedAtTime': str(fimExec), 'prov:label': 'Dataframe com dados das DREs trimestrais dos trimestres 1, 2 e 3'})
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-dreitf-trim123-df"], dicionario_atividades["at-obter-trimestres123"])
    
    return dfTemp

def obter_acumulado_trim3(dataframe_itr, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades):
    
    #Salvar hora do inicio da execucao
    inicioExec = datetime.datetime.now()
    
    # criar o df com informações acumuladas até o trimestre 3, 01/07/ANO a 30/09/ANO - df_acm3
    dfTemp = dataframe_itr.loc[lambda dataframe_itr: ((dataframe_itr.DT_REFER.dt.month == 9) & 
                                                    (dataframe_itr.DT_INI_EXERC.dt.month == 1)) &
                             (dataframe_itr.DT_REFER.dt.year == dataframe_itr.DT_INI_EXERC.dt.year)]
    
    #criar a atividade com hora do final da execucao
    fimExec = datetime.datetime.now()
    dicionario_atividades["at-obter-acm3"] = documento_prov.activity("ufrj:obter_acumulado_trim3", inicioExec, fimExec)
    
    #associar a atividade com o agente
    documento_prov.wasAssociatedWith(dicionario_atividades["at-obter-acm3"], dicionario_agentes["ag-drecvm-ipynb"])
    
    #associar a atividade com o dataset que foi lido
    documento_prov.used(dicionario_atividades["at-obter-acm3"],dicionario_entidades["ent-dreitf-setor-df"])
    
    #gerar entidade do detaframe e associar a entidade
    dicionario_entidades["ent-dreitf-acm3-df"] = documento_prov.entity('obinvest:df_acm3', {'prov:generatedAtTime': str(fimExec), 'prov:label': 'Dataframe com valores das DREs acumuladas dos tres primeiros trimestres'})
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-dreitf-acm3-df"], dicionario_atividades["at-obter-acm3"])
    
    return dfTemp
    
def obter_trimestre_4(df_dfp, df_acm3, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades):

    #Salvar hora do inicio da execucao
    inicioExec = datetime.datetime.now()

    # filtrar os informes com data de referência dez - df acm4
    df_acm4 = df_dfp.loc[lambda df_dfp: (df_dfp.DT_REFER.dt.month == 12) & (df_dfp.DT_REFER.dt.year == df_dfp.DT_INI_EXERC.dt.year)]

    df_acm3_cias = df_acm3.drop(['DT_REFER', 'DENOM_CIA', 'GRUPO_DFP', 'MOEDA', 'ESCALA_MOEDA', 'ORDEM_EXERC', 'DT_INI_EXERC', 
                                 'DT_FIM_EXERC', 'DS_CONTA', 'ST_CONTA_FIXA', 'SETOR_ATIV'], axis=1)

    df_trim4 = pd.merge(df_acm4, df_acm3_cias, how='inner', on=['CNPJ_CIA', 'CD_CVM', 'CD_CONTA', 'ANO'], 
                        suffixes=['_acm4','_acm3'])
    
    # calcular o valor do trimestre 4, fazendo a subtração entre acumulado anual e o acumulado do trimestre 3
    df_trim4['RESULTADO'] = df_trim4['VL_CONTA_acm4'] - df_trim4['VL_CONTA_acm3']
    # deletar colunas desnecessárias
    df_trim4 = df_trim4.drop(['VL_CONTA_acm4', 'VL_CONTA_acm3'], axis=1)
    # renomear as colunas para a concatenação com o df trim123
    dtTemp = df_trim4.rename(columns = {'RESULTADO':'VL_CONTA'})

    # estabelece a data de início do quarto trimestre 01/10/ANO
    dtTemp['DT_INI_EXERC'] = pd.to_datetime(dtTemp['DT_INI_EXERC']).apply(lambda t: t.replace(month=10))

    #criar a atividade com hora do final da execucao
    fimExec = datetime.datetime.now()
    dicionario_atividades["at-obter-trim4"] = documento_prov.activity("ufrj:obter_trimestre_4", inicioExec, fimExec)
    
    #associar a atividade com o agente
    documento_prov.wasAssociatedWith(dicionario_atividades["at-merge-dredfp-setor"], dicionario_agentes["ag-drecvm-ipynb"])
    
    #associar a atividade com o dataset que foi lido
    documento_prov.used(dicionario_atividades["at-obter-trim4"],dicionario_entidades["ent-dredfp-setor-df"])
    documento_prov.used(dicionario_atividades["at-obter-trim4"],dicionario_entidades["ent-dreitf-acm3-df"])
    
    #gerar entidade do detaframe e associar a entidade
    dicionario_entidades["ent-dreirt-trim4-df"] = documento_prov.entity('obinvest:df_trim4', {'prov:generatedAtTime': str(fimExec), 'prov:label': 'Dataframe com dados das DREs do 4 trimestre, subtraindo DFP anual pelo acumulado dos tres primeiros'})
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-dreirt-trim4-df"], dicionario_atividades["at-obter-trim4"])    
    
    return dtTemp

def concatena_trimestres(df_trim123, df_trim4, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades):
    
    #Salvar hora do inicio da execucao
    inicioExec = datetime.datetime.now()

    # cria filtro para empresas que constam tanto no df trim4 quanto no df trim123
    df_trim4_temp = df_trim4.drop(['DT_REFER', 'DENOM_CIA', 'GRUPO_DFP', 'MOEDA', 'ESCALA_MOEDA', 'ORDEM_EXERC', 'DT_INI_EXERC', 
                                             'DT_FIM_EXERC', 'DS_CONTA', 'VL_CONTA', 'ST_CONTA_FIXA', 'SETOR_ATIV'], axis=1)

    df_trim123 = pd.merge(df_trim123, df_trim4_temp, how = 'inner', on = ['CNPJ_CIA', 'CD_CVM', 'CD_CONTA', 'ANO']).reset_index(drop=True)

    dfTemp = pd.concat([df_trim123, df_trim4])
    
    #criar a atividade com hora do final da execucao
    fimExec = datetime.datetime.now()
    dicionario_atividades["at-concat-trim"] = documento_prov.activity("ufrj:concatena_trimestres", inicioExec, fimExec)
    
    #associar a atividade com o agente
    documento_prov.wasAssociatedWith(dicionario_atividades["at-concat-trim"], dicionario_agentes["ag-drecvm-ipynb"])
    
    #associar a atividade com o dataset que foi lido
    documento_prov.used(dicionario_atividades["at-concat-trim"],dicionario_entidades["ent-dreirt-trim4-df"])
    documento_prov.used(dicionario_atividades["at-concat-trim"],dicionario_entidades["ent-dreitf-trim123-df"])
    
    #gerar entidade do detaframe e associar a entidade
    dicionario_entidades["ent-dreirt-trim1234-df"] = documento_prov.entity('obinvest:df_trim1234', {'prov:generatedAtTime': str(fimExec), 'prov:label': 'Dataframe com dados das DREs de todos os trimestres'})
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-dreirt-trim1234-df"], dicionario_atividades["at-concat-trim"])    
    
    
    return dfTemp

def remove_registros_duplicados(df_obinvest_trim1234, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades):
    
    # comparar registros antes e depois
    print('Antes: Linhas = {0} | Colunas = {1}'.format(df_obinvest_trim1234.shape[0], df_obinvest_trim1234.shape[1]))
    
    #Salvar hora do inicio da execucao
    inicioExec = datetime.datetime.now()
    
    dfTemp = df_obinvest_trim1234.drop_duplicates(subset = ['CNPJ_CIA', 'CD_CVM', 'CD_CONTA', 'DT_INI_EXERC', 'DT_FIM_EXERC', 'ANO'], 
                              keep = 'last')

    #criar a atividade com hora do final da execucao
    fimExec = datetime.datetime.now()
    dicionario_atividades["at-remove-dup"] = documento_prov.activity("ufrj:remove_registros_duplicados", inicioExec, fimExec)
    
    #associar a atividade com o agente
    documento_prov.wasAssociatedWith(dicionario_atividades["at-remove-dup"], dicionario_agentes["ag-drecvm-ipynb"])
    
    #associar a atividade com o dataset que foi lido
    documento_prov.used(dicionario_atividades["at-remove-dup"],dicionario_entidades["ent-dreirt-trim1234-df"])
    
    #gerar entidade do detaframe e associar a entidade
    dicionario_entidades["ent-dreitf-tratado-df"] = documento_prov.entity('obinvest:df_obinvest_tratado', {'prov:generatedAtTime': str(fimExec), 'prov:label': 'Dataframe com valores das DREs acumuladas dos tres primeiros trimestres'})
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-dreitf-tratado-df"], dicionario_atividades["at-remove-dup"])
    
    print('Depois: Linhas = {0} | Colunas = {1}'.format(dfTemp.shape[0], dfTemp.shape[1]))
    
    return dfTemp

def adiciona_campo_trimestre(dataframe_dre, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades):
    
    #Salvar hora do inicio da execucao
    inicioExec = datetime.datetime.now()
    
    trimestre =[]
    
    #adicionando as informacoes da coluna de trimestre
    for i in dataframe_dre.itertuples():
      if i.DT_REFER.month == 3:
        trimestre.append(1)
      elif i.DT_REFER.month == 6:
        trimestre.append(2)
      elif i.DT_REFER.month == 9:
        trimestre.append(3)
      else:
        trimestre.append(4)
    
    dataframe_dre = dataframe_dre.assign(TRIMESTRE=trimestre)
    dataframe_dre.reset_index(drop=True)
    
    #criar a atividade com hora do final da execucao
    fimExec = datetime.datetime.now()
    dicionario_atividades["at-adicionar-trimestre"] = documento_prov.activity("ufrj:adiciona_campo_trimestre", inicioExec, fimExec)
    
    #associar a atividade com o agente
    documento_prov.wasAssociatedWith(dicionario_atividades["at-adicionar-trimestre"], dicionario_agentes["ag-drecvm-ipynb"])
    
    #associar a atividade com os dataframes que foram lido
    documento_prov.used(dicionario_atividades["at-adicionar-trimestre"],dicionario_entidades["ent-dreitf-tratado-df"])
    
    #gerar entidade do detaframe e associar a atividade
    dicionario_entidades["ent-dreitr-trimestre-df"] = documento_prov.entity('obinvest:df_obinvest_campo_trimestre', {'prov:generatedAtTime': str(fimExec), 'prov:label': 'Dataframe com dados das DREs trimestrais, adicionada a coluna trimestre'})
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-dreitr-trimestre-df"], dicionario_atividades["at-adicionar-trimestre"])            
    
    return dataframe_dre


def padronizacao_valor_conta(dataframe_dre, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades):

    #Salvar hora do inicio da execucao
    inicioExec = datetime.datetime.now()
    
    # PADRONIZAÇÃO DA COLUNA VL_CONTA
    # dividir por 1000 os valores em que a escala moeda é unidade
    dataframe_dre['VL_CONTA'] = dataframe_dre.apply(lambda x: x.VL_CONTA/1000 if x.ESCALA_MOEDA == 'UNIDADE' 
                                         else x.VL_CONTA, axis = 1)
    
    # renomear os dados da coluna escala moeda para o valor mil
    dataframe_dre['ESCALA_MOEDA'] = dataframe_dre.ESCALA_MOEDA.replace('UNIDADE', 'MIL')
    
    #criar a atividade com hora do final da execucao
    fimExec = datetime.datetime.now()
    dicionario_atividades["at-padr-valor"] = documento_prov.activity("ufrj:padronizacao_valor_conta", inicioExec, fimExec)
    
    #associar a atividade com o agente
    documento_prov.wasAssociatedWith(dicionario_atividades["at-padr-valor"], dicionario_agentes["ag-drecvm-ipynb"])
    
    #associar a atividade com os dataframes que foram lido
    documento_prov.used(dicionario_atividades["at-padr-valor"],dicionario_entidades["ent-dreitr-trimestre-df"])
    
    #gerar entidade do detaframe e associar a atividade
    dicionario_entidades["ent-obinvest-df"] = documento_prov.entity('obinvest:df_obinvest', {'prov:generatedAtTime': str(fimExec), 'prov:label': 'Dataframe final com dados das DREs trimestrais'})
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-obinvest-df"], dicionario_atividades["at-padr-valor"])            
        
    
    return dataframe_dre

def criar_dataset_obinvest(df_obinvest, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades):

    #Salvar hora do inicio da execucao
    inicioExec = datetime.datetime.now()
    
    df_obinvest.to_csv('obinvest-dre-historico.csv', sep=';', encoding='"ISO-8859-1', index=False)
    
    #criar a atividade com hora do final da execucao
    fimExec = datetime.datetime.now()
    dicionario_atividades["at-cria-ds"] = documento_prov.activity("ufrj:cria_dataset_obinvest", inicioExec, fimExec)
    
    #associar a atividade com o agente
    documento_prov.wasAssociatedWith(dicionario_atividades["at-cria-ds"], dicionario_agentes["ag-drecvm-ipynb"])
    
    #associar a atividade com os dataframes que foram lido
    documento_prov.used(dicionario_atividades["at-cria-ds"],dicionario_entidades["ent-obinvest-df"])
    
    #gerar entidade do detaframe e associar a atividade
    dicionario_entidades["ent-obinvest-ds"] = documento_prov.entity('obinvest:ds_obinvest', {'prov:generatedAtTime': str(fimExec), 'prov:label': 'Dataset final com historico de dados das DREs trimestrais', 'prov:type': 'void:Dataset'})
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-obinvest-ds"], dicionario_atividades["at-cria-ds"])       


# faz a limpeza dos datasets cad_cia, dre_itr e dre_dfp e retorna o dataset obinvest
def criar_dataframe_obinvest(documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades):
    
    # carrega os datasets 
    df_cia, df_itr, df_dfp = carregar_datasets(documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades)

    #adicionar dados de Ano
    df_itr_ano, df_dfp_ano = adiciona_ano_dres(df_itr, df_dfp, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades)
    
    #Tratando dados de companhia
    df_cia_tratado = trata_dados_companhias(df_cia, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades)
    
    # fazer merge entre cadastro e informe itr, obtendo o df itr com setor - dfs cad_cia e itr
    df_itr_setor = merge_dre_itr_setor(df_itr_ano, df_cia_tratado, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades)

    #Filtrando dados dos 3 primeiros trimestres 
    df_trim123 = obter_trimestres_123(df_itr_setor, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades)

    # OBTENÇÃO DO DF ACUMULADO TRIMESTRE 3
    df_acm3 = obter_acumulado_trim3(df_itr_setor, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades)
    
    # FATIAMENTO DO DF DFP
    # filtrar últimos informes anuais - df dfp
    df_dfp_ultimo_exerc = filtrar_ultimo_exerc_dfp(df_dfp_ano, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades)
    
    # fazer merge entre cadastro e informe itr, obtendo o df dfp com setor - dfs cad_cia e dfp
    df_dfp_setor = merge_dre_dfp_setor(df_dfp_ultimo_exerc, df_cia_tratado, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades)
    
    # OBTENÇÃO DO DF ACUMULADO TRIMESTRE 4
    df_trim4 = obter_trimestre_4(df_dfp_setor, df_acm3, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades)
    
    # CRIAÇÃO DO DF OBINVEST
    # criar o df obinvest, concatenando os informes referentes aos 4 trimestres
    df_obinvest_trim1234 = concatena_trimestres(df_trim123, df_trim4, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades)
    
    # remove os registros duplicados
    df_obinvest_tratado = remove_registros_duplicados(df_obinvest_trim1234, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades)
    
    #Adicionar dado de trimestre
    df_obinvest_campo_trimestre = adiciona_campo_trimestre(df_obinvest_tratado, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades)
    
    #padronizacao valores contas
    df_obinvest = padronizacao_valor_conta(df_obinvest_campo_trimestre, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades)
    
    #criar dataset obinvest
    criar_dataset_obinvest(df_obinvest, documento_prov, dicionario_agentes , dicionario_atividades, dicionario_entidades)
    
    return df_obinvest

## **Proveniência de Dados**

In [120]:
import sys, subprocess, datetime
from prov.model import ProvDocument, Namespace
from prov.dot import prov_to_dot
from IPython.display import Image

def gerar_saidas_proveniencia(documento_prov):
    entity = "DRE-CVM-PROV"
    #Generating the outup - a  Provenance Graph
    dot = prov_to_dot(documento_prov)
    graph = entity+".png"
    dot.write_png(graph)

    #Generating the Serialization - Output XML
    documento_prov.serialize(entity + ".xml", format='xml') 

    #Generating the Serialization - Output Turtle
    documento_prov.serialize(entity + ".ttl", format='rdf', rdf_format='ttl')

def adicionar_namespaces(documento_prov):
    # Declaring namespaces for various prefixes used in the excution of Randon Walk Experiment
    documento_prov.add_namespace('foaf', 'http://xmlns.com/foaf/0.1/')
    documento_prov.add_namespace('prov', 'http://www.w3.org/ns/prov#')
    documento_prov.add_namespace('void', 'http://vocab.deri.ie/void#')
    documento_prov.add_namespace('ufrj', 'https://www.ufrj.br')
    documento_prov.add_namespace('cvm', 'https://www.gov.br/cvm/pt-br')
    documento_prov.add_namespace('cvm-cademp', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/CAD/DADOS/')
    documento_prov.add_namespace('cvm-dre-itr', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/')
    documento_prov.add_namespace('cvm-dre-dfp', 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/')
    documento_prov.add_namespace('obinvest', 'https://www.obinvest.org')
    return documento_prov

def criar_agentes(documento_prov):
    
    #criando agentes
    dagnts={}
    dagnts["ag-cvm"] = documento_prov.agent("cvm:CVM", {"prov:type":"prov:Organization", "foaf:name":"Comissão de Valores Mobiliários"})
    dagnts["ag-ufrj"] = documento_prov.agent("ufrj:UFRJ", {"prov:type":"prov:Organization", "foaf:name":"Universidade Federal do Rio de Janeiro"})
    dagnts["ag-ppgi"] = documento_prov.agent("ufrj:PPGI", {"prov:type":"prov:Organization", "foaf:name":"Programa de Pós Graduação em Informática"})
    dagnts["ag-mai712"] = documento_prov.agent("ufrj:MAI712", {"prov:type":"prov:Organization", "foaf:name":"Disciplina de Fundamentos de Ciências de Dados"})
    dagnts["ag-grupo2"] = documento_prov.agent("ufrj:GRUPO2", {"prov:type":"prov:Organization", "foaf:name":"Grupo 02 para o trabalho final"})
    dagnts["ag-aluno-gil"] = documento_prov.agent("ufrj:Saulo", {"prov:type":"prov:Person", "foaf:name":"Gilberto Gil Fidelis Gomes Passos", "foaf:mbox":"gilbertogilfgp@gmail.com"})
    dagnts["ag-aluno-saulo"] = documento_prov.agent("ufrj:Gil", {"prov:type":"prov:Person", "foaf:name":"Saulo Andrade Almeida", "foaf:mbox":"sauloandrade@gmail.com"})
    dagnts["ag-aluno-valquire"] = documento_prov.agent("ufrj:Valquire", {"prov:type":"prov:Person", "foaf:name":"Valquire da Silva de Jesus", "foaf:mbox":"valquirej@gmail.com"})
    dagnts["ag-drecvm-ipynb"] = documento_prov.agent("ufrj:drecvm.ipynb", {"prov:type":"prov:SoftwareAgent", "foaf:name":"drecvm.ipynb", "prov:label":"Notebook Python utilizado no trabalho"})

    return dagnts

def associar_agentes_ufrj(dicionario_agentes):
    dicionario_agentes["ag-ppgi"].actedOnBehalfOf(dicionario_agentes["ag-ufrj"])
    dicionario_agentes["ag-mai712"].actedOnBehalfOf(dicionario_agentes["ag-ppgi"])
    dicionario_agentes["ag-grupo2"].actedOnBehalfOf(dicionario_agentes["ag-mai712"])
    dicionario_agentes["ag-drecvm-ipynb"].actedOnBehalfOf(dicionario_agentes["ag-grupo2"])
    dicionario_agentes["ag-aluno-gil"].actedOnBehalfOf(dicionario_agentes["ag-grupo2"])
    dicionario_agentes["ag-aluno-saulo"].actedOnBehalfOf(dicionario_agentes["ag-grupo2"])
    dicionario_agentes["ag-aluno-valquire"].actedOnBehalfOf(dicionario_agentes["ag-grupo2"])

def criar_atividades_iniciais(documento_prov):
    #criando atividades
    dataDownloadDatasets = datetime.datetime.strptime('15/09/22', '%d/%m/%y')
    
    dativs={}
    dativs["at-criar-ds"] = documento_prov.activity("cvm:criar-dataset")
    dativs["at-trabalho-final"] = documento_prov.activity("cvm:trabalho-final")
    dativs["at-armazenar-datasets"] = documento_prov.activity("ufrj:armazenar-datasets", dataDownloadDatasets, None)
    return dativs
    
def criar_entidades_iniciais(documento_prov):
    #criando atividades
    dents={}
    dents["ent-cademp-ds"] = documento_prov.entity('cvm-cademp:cad_cia_aberta.csv', {'prov:label': 'Dataset com dados cadastrais das empresas listadas na CVM', 'prov:type': 'void:Dataset'})
    
    #gerando estrutura dos dados da DRE tipo ITR
    dents["ent-dreitr"] = documento_prov.entity('cvm:dre-itr', {'prov:label': 'Documento que representa o conceito de DREs do tipo Trimestral', 'prov:type': 'foaf:Document'})
    dents["ent-dreitr2011-zip"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_2011.zip', {'prov:label': 'ZIP com Dataset com DRE trimestrais, do ano de 2011', 'prov:type': 'foaf:Document'})
    dents["ent-dreitr2011-ds"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_DRE_con_2011.csv', {'prov:label': 'Dataset com dados de DRE trimestrais, do ano de 2011', 'prov:type': 'void:Dataset'})
    dents["ent-dreitr2012-zip"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_2012.zip', {'prov:label': 'ZIP com Dataset com DRE trimestrais, do ano de 2012', 'prov:type': 'foaf:Document'})
    dents["ent-dreitr2012-ds"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_DRE_con_2012.csv', {'prov:label': 'Dataset com dados de DRE trimestrais,  do anos de 2012', 'prov:type': 'void:Dataset'})
    dents["ent-dreitr2013-zip"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_2013.zip', {'prov:label': 'ZIP com Dataset com DRE trimestrais, do ano de 2013', 'prov:type': 'foaf:Document'})
    dents["ent-dreitr2013-ds"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_DRE_con_2013.csv', {'prov:label': 'Dataset com dados de DRE trimestrais, do ano de 2013', 'prov:type': 'void:Dataset'})
    dents["ent-dreitr2014-zip"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_2014.zip', {'prov:label': 'ZIP com Dataset com DRE trimestrais, do ano de 2014', 'prov:type': 'foaf:Document'})
    dents["ent-dreitr2014-ds"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_DRE_con_2014.csv', {'prov:label': 'Dataset com dados de DRE trimestrais,  do anos de 2014', 'prov:type': 'void:Dataset'})
    dents["ent-dreitr2015-zip"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_2015.zip', {'prov:label': 'ZIP com Dataset com DRE trimestrais, do ano de 2015', 'prov:type': 'foaf:Document'})
    dents["ent-dreitr2015-ds"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_DRE_con_2015.csv', {'prov:label': 'Dataset com dados de DRE trimestrais, do ano de 2015', 'prov:type': 'void:Dataset'})
    dents["ent-dreitr2016-zip"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_2016.zip', {'prov:label': 'ZIP com Dataset com DRE trimestrais, do ano de 2016', 'prov:type': 'foaf:Document'})
    dents["ent-dreitr2016-ds"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_DRE_con_2016.csv', {'prov:label': 'Dataset com dados de DRE trimestrais, do ano de 2016', 'prov:type': 'void:Dataset'})
    dents["ent-dreitr2017-zip"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_2017.zip', {'prov:label': 'ZIP com Dataset com DRE trimestrais, do ano de 2017', 'prov:type': 'foaf:Document'})
    dents["ent-dreitr2017-ds"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_DRE_con_2017.csv', {'prov:label': 'Dataset com dados de DRE trimestrais,  do anos de 2017', 'prov:type': 'void:Dataset'})
    dents["ent-dreitr2018-zip"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_2018.zip', {'prov:label': 'ZIP com Dataset com DRE trimestrais, do ano de 2018', 'prov:type': 'foaf:Document'})
    dents["ent-dreitr2018-ds"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_DRE_con_2018.csv', {'prov:label': 'Dataset com dados de DRE trimestrais, do ano de 2018', 'prov:type': 'void:Dataset'})
    dents["ent-dreitr2019-zip"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_2019.zip', {'prov:label': 'ZIP com Dataset com DRE trimestrais, do ano de 2019', 'prov:type': 'foaf:Document'})
    dents["ent-dreitr2019-ds"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_DRE_con_2019.csv', {'prov:label': 'Dataset com dados de DRE trimestrais,  do anos de 2019', 'prov:type': 'void:Dataset'})
    dents["ent-dreitr2020-zip"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_2020.zip', {'prov:label': 'ZIP com Dataset com DRE trimestrais, do ano de 2020', 'prov:type': 'foaf:Document'})
    dents["ent-dreitr2020-ds"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_DRE_con_2020.csv', {'prov:label': 'Dataset com dados de DRE trimestrais, do ano de 2020', 'prov:type': 'void:Dataset'})
    dents["ent-dreitr2021-zip"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_2021.zip', {'prov:label': 'ZIP com Dataset com DRE trimestrais, do ano de 2021', 'prov:type': 'foaf:Document'})
    dents["ent-dreitr2021-ds"] = documento_prov.entity('cvm-dre-itr:itr_cia_aberta_DRE_con_2021.csv', {'prov:label': 'Dataset com dados de DRE trimestrais, do ano de 2021', 'prov:type': 'void:Dataset'})

    #gerando estrutura dos dados da DRE tipo dfp
    dents["ent-dredfp"] = documento_prov.entity('cvm:dre-dfp', {'prov:label': 'Documento que representa o conceito de DREs do tipo Trimestral', 'prov:type': 'foaf:Document'})
    dents["ent-dredfp2011-zip"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_2011.zip', {'prov:label': 'ZIP com Dataset com DRE anual de 2011', 'prov:type': 'foaf:Document'})
    dents["ent-dredfp2011-ds"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_DRE_con_2011.csv', {'prov:label': 'Dataset com dados de DRE anual de 2011', 'prov:type': 'void:Dataset'})
    dents["ent-dredfp2012-zip"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_2012.zip', {'prov:label': 'ZIP com Dataset com DRE anual de 2012', 'prov:type': 'foaf:Document'})
    dents["ent-dredfp2012-ds"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_DRE_con_2012.csv', {'prov:label': 'Dataset com dados de DRE anual,  do anos de 2012', 'prov:type': 'void:Dataset'})
    dents["ent-dredfp2013-zip"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_2013.zip', {'prov:label': 'ZIP com Dataset com DRE anual de 2013', 'prov:type': 'foaf:Document'})
    dents["ent-dredfp2013-ds"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_DRE_con_2013.csv', {'prov:label': 'Dataset com dados de DRE anual de 2013', 'prov:type': 'void:Dataset'})
    dents["ent-dredfp2014-zip"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_2014.zip', {'prov:label': 'ZIP com Dataset com DRE anual de 2014', 'prov:type': 'foaf:Document'})
    dents["ent-dredfp2014-ds"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_DRE_con_2014.csv', {'prov:label': 'Dataset com dados de DRE anual,  do anos de 2014', 'prov:type': 'void:Dataset'})
    dents["ent-dredfp2015-zip"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_2015.zip', {'prov:label': 'ZIP com Dataset com DRE anual de 2015', 'prov:type': 'foaf:Document'})
    dents["ent-dredfp2015-ds"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_DRE_con_2015.csv', {'prov:label': 'Dataset com dados de DRE anual de 2015', 'prov:type': 'void:Dataset'})
    dents["ent-dredfp2016-zip"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_2016.zip', {'prov:label': 'ZIP com Dataset com DRE anual de 2016', 'prov:type': 'foaf:Document'})
    dents["ent-dredfp2016-ds"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_DRE_con_2016.csv', {'prov:label': 'Dataset com dados de DRE anual de 2016', 'prov:type': 'void:Dataset'})
    dents["ent-dredfp2017-zip"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_2017.zip', {'prov:label': 'ZIP com Dataset com DRE anual de 2017', 'prov:type': 'foaf:Document'})
    dents["ent-dredfp2017-ds"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_DRE_con_2017.csv', {'prov:label': 'Dataset com dados de DRE anual,  do anos de 2017', 'prov:type': 'void:Dataset'})
    dents["ent-dredfp2018-zip"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_2018.zip', {'prov:label': 'ZIP com Dataset com DRE anual de 2018', 'prov:type': 'foaf:Document'})
    dents["ent-dredfp2018-ds"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_DRE_con_2018.csv', {'prov:label': 'Dataset com dados de DRE anual de 2018', 'prov:type': 'void:Dataset'})
    dents["ent-dredfp2019-zip"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_2019.zip', {'prov:label': 'ZIP com Dataset com DRE anual de 2019', 'prov:type': 'foaf:Document'})
    dents["ent-dredfp2019-ds"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_DRE_con_2019.csv', {'prov:label': 'Dataset com dados de DRE anual de 2019', 'prov:type': 'void:Dataset'})
    dents["ent-dredfp2020-zip"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_2020.zip', {'prov:label': 'ZIP com Dataset com DRE anual de 2020', 'prov:type': 'foaf:Document'})
    dents["ent-dredfp2020-ds"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_DRE_con_2020.csv', {'prov:label': 'Dataset com dados de DRE anual de 2020', 'prov:type': 'void:Dataset'})
    dents["ent-dredfp2021-zip"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_2021.zip', {'prov:label': 'ZIP com Dataset com DRE anual de 2021', 'prov:type': 'foaf:Document'})
    dents["ent-dredfp2021-ds"] = documento_prov.entity('cvm-dre-dfp:dfp_cia_aberta_DRE_con_2021.csv', {'prov:label': 'Dataset com dados de DRE anual de 2021', 'prov:type': 'void:Dataset'})
    
    dents["ent-git-obinvest"] = documento_prov.entity('obinvest:github-drecvm', {'prov:label': 'Repositorio DRECVM da OBInvest', 'prov:type': 'prov:Collection'})
    
    return dents
    
def associar_agentes_atividades_entidades_iniciais(documento_prov, dicionario_agentes, 
                                                   dicionario_atividades, dicionario_entidades):
    
    #Associar a atividade de gerar dataset com o agente CVM
    documento_prov.wasAssociatedWith(dicionario_atividades["at-criar-ds"], 
                                     dicionario_agentes["ag-cvm"])
    
    #Associando os datasets com a atividade de gerar dataset da CVM
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-cademp-ds"], dicionario_atividades["at-criar-ds"])
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-dreitr"], dicionario_atividades["at-criar-ds"])    
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-dredfp"], dicionario_atividades["at-criar-ds"])
    
    #Associando ZIPs das DREs ITR e DFP com as entidaes de Datasets genericos
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2011-zip"], dicionario_entidades["ent-dreitr"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2012-zip"], dicionario_entidades["ent-dreitr"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2013-zip"], dicionario_entidades["ent-dreitr"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2014-zip"], dicionario_entidades["ent-dreitr"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2015-zip"], dicionario_entidades["ent-dreitr"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2016-zip"], dicionario_entidades["ent-dreitr"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2017-zip"], dicionario_entidades["ent-dreitr"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2018-zip"], dicionario_entidades["ent-dreitr"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2019-zip"], dicionario_entidades["ent-dreitr"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2020-zip"], dicionario_entidades["ent-dreitr"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2021-zip"], dicionario_entidades["ent-dreitr"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2011-zip"], dicionario_entidades["ent-dredfp"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2012-zip"], dicionario_entidades["ent-dredfp"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2013-zip"], dicionario_entidades["ent-dredfp"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2014-zip"], dicionario_entidades["ent-dredfp"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2015-zip"], dicionario_entidades["ent-dredfp"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2016-zip"], dicionario_entidades["ent-dredfp"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2017-zip"], dicionario_entidades["ent-dredfp"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2018-zip"], dicionario_entidades["ent-dredfp"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2019-zip"], dicionario_entidades["ent-dredfp"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2020-zip"], dicionario_entidades["ent-dredfp"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2021-zip"], dicionario_entidades["ent-dredfp"])  
    
    #Associando ZIPs das DREs ITR e DFP com os CSVs
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2011-ds"], dicionario_entidades["ent-dreitr2011-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2012-ds"], dicionario_entidades["ent-dreitr2012-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2013-ds"], dicionario_entidades["ent-dreitr2013-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2014-ds"], dicionario_entidades["ent-dreitr2014-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2015-ds"], dicionario_entidades["ent-dreitr2015-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2016-ds"], dicionario_entidades["ent-dreitr2016-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2017-ds"], dicionario_entidades["ent-dreitr2017-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2018-ds"], dicionario_entidades["ent-dreitr2018-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2019-ds"], dicionario_entidades["ent-dreitr2019-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2020-ds"], dicionario_entidades["ent-dreitr2020-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2021-ds"], dicionario_entidades["ent-dreitr2021-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2011-ds"], dicionario_entidades["ent-dredfp2011-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2012-ds"], dicionario_entidades["ent-dredfp2012-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2013-ds"], dicionario_entidades["ent-dredfp2013-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2014-ds"], dicionario_entidades["ent-dredfp2014-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2015-ds"], dicionario_entidades["ent-dredfp2015-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2016-ds"], dicionario_entidades["ent-dredfp2016-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2017-ds"], dicionario_entidades["ent-dredfp2017-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2018-ds"], dicionario_entidades["ent-dredfp2018-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2019-ds"], dicionario_entidades["ent-dredfp2019-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2020-ds"], dicionario_entidades["ent-dredfp2020-zip"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2021-ds"], dicionario_entidades["ent-dredfp2021-zip"])    
    
    #associar a atividade do trabalho final, com os agentes da dicilpina e da equipe
    documento_prov.wasAssociatedWith(dicionario_atividades["at-trabalho-final"], dicionario_agentes["ag-mai712"])
    documento_prov.wasAssociatedWith(dicionario_atividades["at-trabalho-final"], dicionario_agentes["ag-grupo2"])   

    #associar agente do notebook com a atividade do trabalho final
    documento_prov.wasAssociatedWith(dicionario_atividades["at-trabalho-final"], dicionario_agentes["ag-drecvm-ipynb"])    
    
    #associar atividade de armazenar Dataset com o agente grupo2
    documento_prov.wasAssociatedWith(dicionario_atividades["at-armazenar-datasets"], dicionario_agentes["ag-grupo2"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-cademp-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dreitr2011-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dreitr2012-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dreitr2013-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dreitr2014-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dreitr2015-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dreitr2016-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dreitr2017-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dreitr2018-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dreitr2019-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dreitr2020-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dreitr2021-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dredfp2011-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dredfp2012-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dredfp2013-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dredfp2014-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dredfp2015-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dredfp2016-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dredfp2017-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dredfp2018-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dredfp2019-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dredfp2020-ds"])
    documento_prov.used(dicionario_atividades["at-armazenar-datasets"],dicionario_entidades["ent-dredfp2021-ds"])
    
    #associar repositorio github da obinvest a atividade de armazenar datasets
    documento_prov.wasGeneratedBy(dicionario_entidades["ent-git-obinvest"], dicionario_atividades["at-armazenar-datasets"])
    
    #Detalhamento dos csvs no github
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-cademp-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2011-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2012-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2013-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2014-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2015-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2016-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2017-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2018-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2019-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2020-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dreitr2021-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2011-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2012-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2013-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2014-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2015-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2016-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2017-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2018-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2019-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2020-ds"], dicionario_entidades["ent-git-obinvest"])
    documento_prov.wasDerivedFrom(dicionario_entidades["ent-dredfp2021-ds"], dicionario_entidades["ent-git-obinvest"])
    
def inicializaProveniencia():
    # Creating an empty provenance document
    doc_prov = ProvDocument()

    #criando os namespaces do documento de proveniencia
    doc_prov = adicionar_namespaces(doc_prov)
    
    #criando agentes
    agentes_dic = criar_agentes(doc_prov)
    
    #criando as hierarquia de agentes
    associar_agentes_ufrj(agentes_dic)
    
    #criando atividades iniciais
    atividades_dic = criar_atividades_iniciais(doc_prov)
    
    #criando entidades iniciais
    entidades_dic = criar_entidades_iniciais(doc_prov)
    
    #associacoes inicias da proveniencia
    associar_agentes_atividades_entidades_iniciais(doc_prov, agentes_dic, atividades_dic, entidades_dic)
    
    #criando entidades iniciais
    return doc_prov, agentes_dic, atividades_dic, entidades_dic

## **Análise de Dados**
### **Análise Inicial dos Dados**

In [121]:
def analiseInicialDataset(dataframe_obinvest):
    #print(dataframe_obinvest.info())
    print(dataframe_obinvest.shape)
    #print(dataframe_obinvest.isnull().sum())
    #print(dataframe_obinvest.describe())
    #print(dataframe_obinvest.columns)
    #print(dataframe_obinvest.DT_REFER.value_counts())
    #print(dataframe_obinvest.ESCALA_MOEDA.value_counts())

### **Análise Estatística dos Dados**

In [122]:
import plotly.express as px

def analiseEstatisticaDataset(dataFrameObinvest):
  #print('Rodando experimento de dataset estatisticos')
  # consulta o df obinvest pelas contas 3.01
  df_conta_301 = dataFrameObinvest.query('CD_CONTA == "3.01"')

  # consulta o df obinvest pelas contas 3.09
  df_conta_309 = dataFrameObinvest.query('CD_CONTA == "3.09"')
 
  # agrupar por conta, setor, ano e trimestre
  vetor_301 = df_conta_301.groupby(['SETOR_ATIV', 'DT_REFER'])['VL_CONTA'].sum()
  vetor_309 = df_conta_309.groupby(['SETOR_ATIV', 'DT_REFER'])['VL_CONTA'].sum()

  df_margem_setor = pd.DataFrame(vetor_301)
  df_margem_setor = df_margem_setor.rename(columns = {'VL_CONTA':'SOMA_CONTA_3_01'})
  df_margem_setor['SOMA_CONTA_3_09'] = vetor_309.values
  df_margem_setor['MARGEM_SETOR'] = df_margem_setor.apply(lambda x: 0 if x.SOMA_CONTA_3_01 == 0 else (x.SOMA_CONTA_3_09/x.SOMA_CONTA_3_01) * 100,  axis = 1)
    
  data_ref=[]
  for sub_lista in df_margem_setor.index:
    data_ref.append(sub_lista[1])  # seleciona o trimestre

  setores=[]
  for sub_lista in df_margem_setor.index:
    setores.append(sub_lista[0])  # seleciona o trimestre

  df_margem_setor['DT_REFER'] = data_ref
  df_margem_setor['SETOR'] = setores

  query =  ' SETOR == "Energia Elétrica" | '
  query += ' SETOR == "Bancos" | '
  query += ' SETOR == "Serviços Transporte e Logística" | '
  query += ' SETOR == "Comércio (Atacado e Varejo)" | '
  query += ' SETOR == "Educação" '

  df_margem_setor = df_margem_setor.query(query)

  fig = px.line(df_margem_setor, x="DT_REFER", y=df_margem_setor['MARGEM_SETOR'],
              title='Margem Setor', color='SETOR', markers=True, symbol="SETOR")
  return fig

## **Execução Principal**

In [123]:
def executaExperimento():
    doc_prov, dic_agentes , dic_atividades, dic_entidades = inicializaProveniencia()
    df_obinvest = criar_dataframe_obinvest(doc_prov, dic_agentes , dic_atividades, dic_entidades)
    #analiseInicialDataset(df_obinvest)
    plotMargenSetor = analiseEstatisticaDataset(df_obinvest)
    gerar_saidas_proveniencia(doc_prov)
    return plotMargenSetor

In [126]:
def main():
    if(verifica_versoes_ambiente_bibliotecas()):
        return executaExperimento()
    else:
        print('O experimento não foi executado pois existe algo errado com o ambiente ou versões das bibliotecas!')
        return executaExperimento()

plotMargemSetor = main()

O experimento não foi executado pois existe algo errado com o ambiente ou versões das bibliotecas!
Antes: Linhas = 413641 | Colunas = 16
Depois: Linhas = 413641 | Colunas = 16


In [127]:
plotMargemSetor.show()

In [ ]:
Image("DRE-CVM-PROV.png")